# Лабораторная работа 4.  Алгоритмы на графах

**Цель работы:** изучение некоторых алгоритмов на графах; исследование эффективности этих алгоритмов.

Задание 1: Найти кратчайший путь на графе между парами вершин методом динамического программирования вручную.
| Вариант | Начальная и конечная вершины| Граф |
|:---|:---|:---|
| 6  | 1, 6 | ![](../img/006.png) |

Условия выхода из рекурсии $f_1=0$, $S_{11} = 0$.

тогда расчитаем кратчайшие пути $f_i$ от 1-й до i-й вершины, ведущей к 6:

$f_2 = \min(S_{21} + f_1) = 2 + f_1 = 2$;

$f_3 = \min(S_{31} + f_1) = 13 + f_1 = 13$;

$f_4 = \min 
\left(
  \begin{array}{c}
  S_{41} + f_1 \\
  S_{42} + f_2 \\
  S_{43} + f_3 \\
  \end{array}
\right)= \min 
\left(
  \begin{array}{c}
  25 + 0 \\
  3 + 2 \\
  1 + 13 \\
  \end{array}
\right) = 5$; 

$f_6 = \min 
\left(
  \begin{array}{c}
  S_{63} + f_3 \\
  S_{64} + f_4 \\
  \end{array}
\right)= \min 
\left(
  \begin{array}{c}
  7 + 13 \\
  4 + 5 \\
  \end{array}
\right) = 9$; 

Теперь найдем подходящие пути, равные 9 усл.ед. и соответственно равные оптимальной траектории.

Пусть $f_i = 6$, тогда:

$f_6 = \min 
\left(
  \begin{array}{c}
  S_{63} + f_3 \\
  S_{64} + f_4 \\
  \end{array}
\right)= \min 
\left(
  \begin{array}{c}
  7 + 13 \\
  4 + 5 \\
  \end{array}
\right) = 9$;

$4 + f_4 = 9$ => $f_i = f_4$

$f_4 = \min 
\left(
  \begin{array}{c}
  S_{41} + f_1 \\
  S_{42} + f_2 \\
  S_{43} + f_3 \\
  \end{array}
\right)= \min 
\left(
  \begin{array}{c}
  25 + 0 \\
  3 + 2 \\
  1 + 13 \\
  \end{array}
\right) = 5$; 

$3 + f_2 = 4$ => $f_i = f_2$

из $f_2$ только единственный путь ведет к начальной вершине $f_1$, тогда оптимальная траекотрия от 1 до 6:

1 --> 2 --> 4 --> 6

Задание 2: Реализовать прогрммно поиск кратчайшего пути на графе между парами вершин из задания 1 методом динамического программирования.

In [22]:
# представим граф в виде матрицы смежности весов
# нулевая вершина не связана с остальными

import math

_ = math.inf

G = [
    #0  1  2  3  4  5  6  7
    [0, _, _, _, _, _, _, _], # 0
    [_, 0, 2,13,25,17, _, _], # 1
    [_, _, 0, _, 3, 6, _, _], # 2
    [_, _, _, 0, 1, _, 7, _], # 3
    [_, _, _, _, 0, _, 4,35], # 4
    [_, _, _, _, _, 0, _,20], # 5
    [_, _, _, _, _, _, 0, 5], # 6
    [_, _, _, _, _, _, _, 0], # 7
]

def find_min_route(G, start, end):
    N = len(G)
    D = [[G[i][j] for j in range(N)] for i in range(N)]
    M = [[i for j in range(N)] for i in range(N)]
    for k in range(N):
        for i in range(N):
            for j in range(N):
                if D[i][k] + D[k][j] < D[i][j]:
                    D[i][j] = D[i][k] + D[k][j]
                    M[i][j] = M[k][j]
    
    route = [end]
    while end != start:
        end = M[start][end]
        route.append(end)
    return route[::-1]

find_min_route(G, 1, 6)

[1, 2, 4, 6]

Задание 3. Реализовать алгоритм Дейкстры поиска кратчайшего пути на графе между парами вершин:
| Вариант | Начальная и конечная вершины| Граф | 
|:---|:---|:---|
| 6  | 3, 8 | ![](../img/016.png) |

In [19]:
import math

_ = math.inf

G = [
    #0  1  2  3  4  5  6  7  8
    [0, _, _, _, _, _, _, _, _], # 0
    [_, 0,14, _, _, _, 8, _, _], # 1
    [_,14, 0, _,10, 2, 2, _, 9], # 2
    [_, _, _, 0, _, _, _, _,11], # 3
    [_, _,10, _, 0, 3, 6, 5, _], # 4
    [_, _, 2, _, 3, 0, _, 8, 1], # 5
    [_, 8, 2, _, 6, _, 0, 5, _], # 6
    [_, _, _, _, 5, 8, 5, 0, 7], # 7
    [_, _, 9,11, _, 1, _, 7, 0], # 8
]

In [20]:
def arg_min(T, S):
    amin = -1
    m = math.inf
    for i, t in enumerate(T):
        if t < m and i not in S:
            m = t
            amin = i
    return amin

# Алгоритм Дейкстры для двух вершин графа
def deikstra_alg(G, start, end):
    N = len(G)
    T = [math.inf] * N  
    v = start       
    used = {v}     
    T[v] = 0    
    M = [0] * N 
    while v != -1:          
        for j, dw in enumerate(G[v]):   
            if j not in used:           
                w = T[v] + dw
                if w < T[j]:
                    T[j] = w
                    M[j] = v        
        v = arg_min(T, used)            
        if v >= 0:                    
            used.add(v)               

    route = [end]
    while end != start:
        end = M[route[-1]]
        route.append(end)
    return route[::-1]
    
deikstra_alg(G, 3, 8)

[3, 8]

Задание 4: Реализовать прогрммно один из алгоритмов поиска кратчайшего пути на графе между парами вершин из задания 3.


In [21]:
# Реализуем алгоритм Беллмана — Форда для двух вершин графа
def bellman_ford_alg(G, start, end):
    N = len(G)
    D = [math.inf] * N
    D[start] = 0
    M = [0] * N
    for i in range(N - 1):
        for u in range(N):
            for v, w in enumerate(G[u]):
                if w is None:
                    continue
                if D[u] + w < D[v]:
                    D[v] = D[u] + w
                    M[v] = u
    for u in range(N):
        for v, w in enumerate(G[u]):
            if w is None:
                continue
            if D[u] + w < D[v]:
                raise ValueError("Negative cycle error!!!")

    route = [end]
    while end != start:
        end = M[route[-1]]
        route.append(end)
    return route[::-1]

bellman_ford_alg(G, 3, 8)

[3, 8]